In [148]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
#required encoding for scraping, otherwise defaults to unicode and screws things up
#import sys;
#reload(sys);
#sys.setdefaultencoding("utf8")

In [149]:
#load URLs of S&P 500 companies
BASE_URL = [
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MMM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ABT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ABBV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ACN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ATVI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AYI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADBE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AAP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AES',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AET',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AFL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=A',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=APD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AKAM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ALK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ALB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AGN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LNT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ALXN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ALLE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ALL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GOOGL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GOOG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMZN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AEE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AAL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AEP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AXP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AIG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AWK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ABC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AME',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMGN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=APH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=APC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ANTM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AON',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=APA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AIV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AAPL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AMAT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ARNC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AJG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AIZ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=T',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADSK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AZO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AVB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AVY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BHI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BLL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BAC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BCR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BAX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BBT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BDX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BBBY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BBY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BIIB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BLK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HRB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BWA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BXP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BSX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BMY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=AVGO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BFb.N',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CHRW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CPB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CAH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HSIC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KMX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CCL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CAT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CBG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CBS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CELG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CNC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CNP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CTL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CERN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SCHW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CHTR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CHK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CVX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CMG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CHD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XEC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CINF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CTAS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CSCO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=C',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CFG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CTXS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CLX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CME',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CMS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CTSH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CMCSA.O',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CMA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CAG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CXO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ED',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=STZ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GLW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COST',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COTY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CCI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CSRA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CSX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CMI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CVS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DHI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DHR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DRI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DVA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DLPH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DAL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XRAY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DVN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DLR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DFS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DISCA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DISCK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DLTR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=D',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DOV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DOW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DPS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DTE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DUK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DNB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ETFC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EMN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ETN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EBAY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ECL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EIX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EMR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ENDP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ETR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EOG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EQT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EFX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EQIX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EQR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ESS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ES',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EXC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EXPE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EXPD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ESRX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=EXR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XOM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FFIV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FAST',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FRT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FDX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FIS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FITB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FSLR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FISV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FLIR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FLS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FLR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FMC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FTI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=F',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FTV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FBHS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=BEN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FCX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FTR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GPS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GRMN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GGP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GIS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GPC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GILD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GPN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=GWW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HAL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HBI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HOG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HAR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HRS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HIG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HAS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HCA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HCP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HES',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HPE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HOLX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HON',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HRL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HST',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HPQ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HUM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HBAN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ITW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ILMN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=INTC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ICE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IBM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IPG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IFF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=INTU',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ISRG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IVZ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=IRM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JEC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JBHT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SJM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JNJ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JCI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JPM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JNPR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KSU',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=K',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KEY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KMB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KIM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KMI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KLAC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KSS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KHC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LLL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LRCX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LEG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LEN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LVLT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LUK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LLY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LNC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LLTC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LKQ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LMT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=L',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LOW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LYB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MTB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MAC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=M',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MNK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MRO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MPC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MAR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MMC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MLM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MAS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MAT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MKC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MCD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MCK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MJN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MDT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MRK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MET',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MTD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=KORS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MCHP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MU',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MSFT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MHK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TAP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MDLZ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MON',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MNST',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MCO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MOS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MSI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MUR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=MYL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NDAQ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NOV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NAVI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NTAP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NFLX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NWL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NFX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NEM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NWSA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NWS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NEE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NLSN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NKE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NBL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=JWN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NSC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NTRS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NOC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NRG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NUE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=NVDA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ORLY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=OXY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=OMC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=OKE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ORCL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=OI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PCAR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PDCO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PAYX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PYPL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PNR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PBCT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PEP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PKI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PRGO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PFE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PCG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PSX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PNW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PXD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PBI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PNC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PPG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PPL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PCLN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PFG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PGR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PLD',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PRU',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PEG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PSA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PHM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PVH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=QRVO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=PWR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=QCOM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DGX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RRC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RTN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=O',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RHT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=REGN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RSG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RAI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RHI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ROK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ROP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ROST',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RCL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=R',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=CRM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SCG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SLB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SNI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=STX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SEE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SRE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SHW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SIG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SPG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SWKS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SLG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SNA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=LUV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SWN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SE',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SPGI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=STJ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SWK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SPLS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SBUX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=STT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SRCL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SYK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=STI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SYMC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SYF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=SYY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TROW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TGT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TEL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TGNA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TDC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TSO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TXN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TXT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=COO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HSY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TRV',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TMO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TIF',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TWX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TJX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TMK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TSS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TSCO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TDG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=RIG',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TRIP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FOXA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=FOX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=TSN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UDR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ULTA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=USB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UA.C',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UNP',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UAL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UNH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UPS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=URI',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UTX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UHS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=UNM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=URBN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VFC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VLO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VAR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VTR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VRSN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VRSK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VZ',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VRTX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VIAB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=V',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VNO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=VMC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WMT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WBA',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=DIS',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WAT',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WFC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=HCN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WDC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WU',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WRK',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WY',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WHR',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WFM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WMB',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WLTW',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WEC',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WYN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=WYNN',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XEL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XRX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XLNX',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=XYL',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=YHOO',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=YUM',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ZBH',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ZION',
'http://www.reuters.com/finance/stocks/financialHighlights?symbol=ZTS',
]

   

In [150]:
#loading empty array for board members
tax_rates = []

for b in BASE_URL:
    html = requests.get(b).text
    soup = BeautifulSoup(html, "html.parser")
#officer_table_all = soup.find_all('td', {"class" : "data"})
    officer_table = soup.find('td', text=re.compile('Effective Tax Rate - 5 Yr. Avg.'))
    sibling = officer_table.findNext('td')
    #clause needed to skip any companies without board member tables
    try:
        for i in officer_table:
            j = i.findNext('td')
            #used to clear the td tags and just extract the tax rate
            k=j.get_text()
            tax_rates.append((b,k))
    except: pass  

In [151]:
tax_rates

[('http://www.reuters.com/finance/stocks/financialHighlights?symbol=MMM',
  '28.56'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=ABT',
  '12.33'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=ABBV',
  '16.79'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=ACN',
  '23.84'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=ATVI',
  '19.95'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=AYI',
  '34.61'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=ADBE',
  '23.81'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=AAP',
  '37.35'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=AES',
  '41.18'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=AET',
  '38.67'),
 ('http://www.reuters.com/finance/stocks/financialHighlights?symbol=AFL',
  '34.16'),
 ('http://www.reuters.com/finance/stocks/financialH

In [152]:
#convert arrary to dataframe
df = pd.DataFrame(tax_rates)

In [153]:
#rename columns, check output
df.columns = ['URL', 'Effective Tax Rate (TTM)']
df.head(10)

,URL,Effective Tax Rate (TTM)
0,http://www.reuters.com/finance/stocks/financia...,28.56
1,http://www.reuters.com/finance/stocks/financia...,12.33
2,http://www.reuters.com/finance/stocks/financia...,16.79
3,http://www.reuters.com/finance/stocks/financia...,23.84
4,http://www.reuters.com/finance/stocks/financia...,19.95
5,http://www.reuters.com/finance/stocks/financia...,34.61
6,http://www.reuters.com/finance/stocks/financia...,23.81
7,http://www.reuters.com/finance/stocks/financia...,37.35
8,http://www.reuters.com/finance/stocks/financia...,41.18
9,http://www.reuters.com/finance/stocks/financia...,38.67


In [154]:
df.to_csv('/Users/erikrood/desktop/tax_rates.csv')